In [1]:
# Okay time for our K-Means algorithm! woop woop!

# We want our Kmeans algorithm not only to cluster by k given cluster, it should also be able to find best k
# this will be done by an approach from the distortion theory, but later more on that. For now we just need to keep
# that in mind for our approach tactic

# 1. We need to take k+1 (starting with k=1) data points with k_max =< amount of values. (k_1+1,...,k_max) 

# 2. Calculate the euclidean distance to all data points and asign each to the class of the closest datapoint
# 3. Calculate each center of mass for this distribution
# 4. Repeat 2 and 3 m times until the centers of mass don't change and then continue with 1

# 5. Now just insert the results in the formula to best k by finding the lowest rate of distortion withing the calculated clusters

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from math import sqrt
from sklearn import datasets
from sklearn.datasets import make_blobs

# Create a clustered dataset to use
ds,y = make_blobs(n_samples=12, n_features=3, centers=4)

# Just to have it also as a dataframe, maybe we want to work with this
# But I'd like to try it as a version of n dimensional numpy arrays, because i used a lot of pandas in the previous exercises
df = pd.DataFrame (ds, y)
df['cluster'] = y
df

,0,1,2,cluster
2,-9.304885,1.691769,1.139202,2
3,-10.427546,-6.744266,-4.038529,3
2,-10.483713,1.495862,0.340748,2
3,-9.012487,-7.452356,-2.472303,3
0,5.448747,-5.410014,2.925210,0
3,-9.461022,-8.210320,-2.963725,3
0,5.539260,-5.085410,3.069483,0
1,-7.291414,-8.699045,9.354215,1
1,-4.967827,-8.683483,8.878855,1
1,-6.518002,-7.515309,10.688842,1


In [3]:
# 1. Get random starting points.
# when the logic on the simple model works, we will just embedd an incrementer on k and then compare the results for different k's
# only important thing till now is, k has to be changable and the code stil needs to work

clusters = []
centroids = {}
k = 3
i=0
l = 0

sample_numbers = random.sample(range(1, len(df)), k)

while i < k:
    centroids[i] = ds[sample_numbers[i]]
    clusters.append(list(centroids[i]))
    i = i + 1                 #else: call next function
    
# fine now we have a list of centroids in our list of clusters
clusters

[[-10.427546200483134, -6.74426640837995, -4.038529247585167],
 [5.539260250655745, -5.085409939456111, 3.0694833784323214],
 [-9.461022168839044, -8.21032017434563, -2.963724598533105]]

In [4]:
def euclidean_distance_on_array(target, distance_to):
    distance = 0.0
    for n in range(0,k-1): 
        distance = distance + (target[n] - distance_to[n])**2
    return sqrt(distance)

euclidean_distance_on_array(centroids[0], ds[3])


1.5823348185787158

In [5]:
def clustering(clusterspoints, dataset):   
    j = 0
    store = []  
    while j < k:     
        for clpo in clusterspoints:
            for row in dataset:
                store.append((row,euclidean_distance_on_array(row,clpo)))
        j = j+1
        return (store)

## Okay we got a list of arrays of the distances and datapoints to each cluster.

## Contining questions: 
###Work on the list, with k as an index for comparisment and then asign them to the center of clusters
###Or Seperate in three different lists by k values and then compare them
###assumption:
####the first approach would spare storage because this doesn't need to copy data and with indexing
####it's also going through the list fast with n/k simple comparisments of k values

#####!!!! With the first approach, storing the date in list_of_clusters needs to be specified by the amount
##### of the position with an if/else condition on: (len(list)/amount(clusters)) = 1, then cluster 0 ...

# Was heavier then expected with keeping it as dynamic as possible (for now)

distanced = clustering(clusters, ds)
distanced

[(array([-9.30488484,  1.69176872,  1.13920151]), 8.51040876023201),
 (array([-10.4275462 ,  -6.74426641,  -4.03852925]), 0.0),
 (array([-10.48371345,   1.49586209,   0.34074824]), 8.240319924852578),
 (array([-9.01248703, -7.45235611, -2.47230333]), 1.5823348185787158),
 (array([ 5.4487465 , -5.4100137 ,  2.92520958]), 15.932259736102042),
 (array([-9.46102217, -8.21032017, -2.9637246 ]), 1.75598472329565),
 (array([ 5.53926025, -5.08540994,  3.06948338]), 16.05274783433106),
 (array([-7.29141424, -8.69904549,  9.35421479]), 3.695468156270527),
 (array([-4.96782721, -8.68348285,  8.87885495]), 5.7938840009020804),
 (array([-6.51800163, -7.51530902, 10.6888415 ]), 3.9848520009643043),
 (array([ 6.97487358, -3.29644711,  1.32252088]), 17.740678461273262),
 (array([-6.6424682 ,  3.00661781,  1.32494601]), 10.459759009071206),
 (array([-9.30488484,  1.69176872,  1.13920151]), 16.318051176794462),
 (array([-10.4275462 ,  -6.74426641,  -4.03852925]), 16.05274783433106),
 (array([-10.4837134

In [94]:
#s = size of an average cluster. I had it calculated inplace before, but on large data it'll spare some calculations i guess
s = (len(distanced)/k)
int(s)
l = 0
dist_to_cluster=[]

stored = {}
while l < k:
    stored[l] = distanced[int(s*l)+int(s-1)]
    l = l+1

##Proof if the maths is correct
#while l < k:
#    print(int((len(distanced)/k)*l),int((len(distanced)/k)*l)+int((len(distanced)/k))-1)
#    print(l)
#    l = l+1
stored

{0: (array([-6.6424682 ,  3.00661781,  1.32494601]), 10.459759009071206),
 1: (array([-6.6424682 ,  3.00661781,  1.32494601]), 14.62448019838062),
 2: (array([-6.6424682 ,  3.00661781,  1.32494601]), 11.565636352224045)}

In [161]:
#Make a quick list of clusters so we can store the closest neighbor easily
list_of_clusters=[]
for row in clusters:
    list_of_clusters.append([row])
list_of_clusters

#we can now append the 'row' of the dictonary with the smalest distanced into the same 'row' as in this list

[[[-10.427546200483134, -6.74426640837995, -4.038529247585167]],
 [[5.539260250655745, -5.085409939456111, 3.0694833784323214]],
 [[-9.461022168839044, -8.21032017434563, -2.963724598533105]]]

In [162]:
## Okay this step was quite tricky and took a while to figure out but now we can extract the closest values and have the cluster aswell
sto_list = [(k,v) for k,v in stored.items()]
sto_list

c = 0
dist_int = []

while c < k:
    if c == k-1:
        list_of_clusters[dist_int[0]].append(dist_int[1][0].tolist())
        break
    else:
        if sto_list[c][-1][-1] < sto_list[c+1][-1][-1]:
            dist_int = sto_list[c]
            c = c+1
        else:
            c = c+1
list_of_clusters
##Super Neet!

[[[-10.427546200483134, -6.74426640837995, -4.038529247585167],
  [-6.64246819721094, 3.006617809832839, 1.3249460109710745]],
 [[5.539260250655745, -5.085409939456111, 3.0694833784323214]],
 [[-9.461022168839044, -8.21032017434563, -2.963724598533105]]]

In [155]:
# Okay now it's time to assemble our components into propper functions

-6.64246819721094

10.459759009071206